#Assignment 4
Feel free to look around.


In [2]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
using CUDAnative, CUDAdrv, CuArrays

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


Also we shall add BenchmarkTools to benchmark our code and Test for testing code.

In [3]:
Pkg.add("BenchmarkTools")
Pkg.add("Test")
using BenchmarkTools, Test

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


I wanted to define the elements globally

In [4]:
N=parse(Int64, readline())
A=rand(Int32, (N,N))
B=rand(Int32, (N,N))
CPar=zeros(Int32, (N,N))
CSeq=zeros(Int32, (N,N))

stdin> 2


2×2 Array{Int32,2}:
 0  0
 0  0

legacy function for taking transpose, so I could supply the iteration to be in only one dimension. That way, I could multiply as

@cuda threads and stuff C[i,j]=A[i, index]*B[i,index]

where B is actually B transposed, thereby travesring with only one x. But then I found you could parallelize in blocks, and that was easier. This is just PoW.

In [0]:
B
for i in 1:N
  for j in 1:N
    Bt[i,j]=B[j,i]
   end
end
Bt

2×2 Array{Int32,2}:
 3  3
 4  3

##Sequential multiplication


In [10]:
function seq_mult!(A,B,N)
    CSq=A*B
    return CSq
end

seq_mult! (generic function with 1 method)

Now making the test

In [11]:
C=A*B
CSeq=@btime seq_mult!(A,B,N)
@test isapprox(C, CSeq)

  54.618 ns (1 allocation: 96 bytes)


Test Passed

##Parallel multiplication with one dimension of parallelisation


The parallelisation in this bit is only in selecting the terms we multiply. We work in blocks, and multiply on and on. The multiplication itself is in parallel.

Making the arrays cuda-ready

In [5]:
d_a=CuArray(A)
d_b=CuArray(B)
d_c=CuArray(CPar)

2×2 CuArray{Int32,2}:
 0  0
 0  0

the final mult!

In [8]:
function mult_par!(a,b,c,N)
    tidx=threadIdx().x
    tidy=threadIdx().y
    bidx=blockIdx().x
    bidy=blockIdx().y
    ind_x= tidx + blockDim().x * (bidx-1)
    ind_y= tidy + blockDim().y * (bidy-1)
    if(ind_x <= N && ind_y <= N)
        sum=0
        for i=1:N
            @inbounds sum=sum+a[ind_x,i]*b[i,ind_y]
        end
        @inbounds c[ind_x,ind_y]=sum
    end
    return
end

mult_par! (generic function with 1 method)

In [12]:
@btime @cuda blocks=( Int(ceil(N/32)),Int(ceil(N/32)) ) threads=(32,32) mult_par!(d_a,d_b,d_c,N)
CPar=Array(d_c);
@test isapprox(C,CPar)

  9.853 μs (65 allocations: 2.48 KiB)


Test Passed

##Parallel multiplication with dual parallelization(WIP)
For this next bit, I've been working on it for some time now, so bear with me as I try to explain this:
I knew we'd be solving for the the elements of the ith row and jth column, so I would parallelise that.
Then I thought, why not parallelise the addition of the terms too? This would blow up the memory exponentially, but would theoretically make the execution quicker. That's where all my time went. But I don't know how to get this to work, something about not calling threads inside other threads. 


![alt text](https://i.imgflip.com/2vslvl.png)

With the arrays defined, I needed to define function  par_mult, so that we could rowwise mult each element.

In [0]:
d_a = CuArray(A)
d_b = CuArray(B)
d_c = CuArray(C);

In [25]:
d_c

2×2 CuArray{Int32,2}:
 981038679  -1035465523
 933358970    332883184

Made a function to find the i-jth term, and defined it to run in parallel with the multiplication itself.

In [26]:
function find_term(d_a, d_b, x, y, term)
    tidx=threadIdx().x
    bidx=blockIdx().x
    x_idx=blockDim().x*(bidx-1)+tidx
    term+=d_a[x, x_idx]*d_b[x_idx, y]
    
end

find_term (generic function with 1 method)

In [29]:
function par_mult!(d_a,d_b,d_c,N)
    x = (blockIdx().x-1)*blockDim().x + threadIdx().x 
    y = (blockIdx().y-1)*blockDim().y + threadIdx().y 
    @cuda blocks=(Int(ceil(N/32)),Int(ceil(N/32)) ) threads=(32,32) find_term(d_a,d_b,x, y, d_c[x,y])
end

par_mult! (generic function with 2 methods)

In [30]:
@btime @cuda blocks=( Int(ceil(n/32)),Int(ceil(n/32)) ) threads=(32,32) par_mult!(d_a,d_b,d_c,N)

┌ Info: Building the CUDAnative run-time library for your sm_70 device, this might take a while...
└ @ CUDAnative /root/.julia/packages/CUDAnative/ytV2j/src/compiler/rtlib.jl:154


CUDAnative.InvalidIRError: ignored

In [0]:
function driverFunc()
    for i in 1:N
        @cuda threads=N runKernel(d_a, d_b, d_c)
    end
end